#### 1478. Allocate Mailboxes

* https://leetcode.com/problems/allocate-mailboxes/description/

#### Bloomberg

In [ ]:
# Preferred solution - Bottom Up DP
# Insightful explanation - https://chatgpt.com/c/6954c396-49cc-8322-88b5-6220d598ee65
# TC - O(nlogn + n^3 + k*n^2) ~ O(k*n^2) as dp is dominant
# SC - O(k*n) # k postoffice into n houses

class Solution:
    def minDistance(self, houses, k: int) -> int:

        n = len(houses)
        houses.sort() # nlogn 
        # cost[i][j] = min distance to serve houses[i..j] with ONE mailbox
        cost = [[0]*n for _ in range(n)]
        # Precompute costs using median property
        for i in range(n):
            for j in range(i, n):
                m = (i+j)//2
                cost[i][j] = sum(abs(houses[x]-houses[m]) for x in range(i, j+1))

        # Step 2 -
        # # dp[m][i] = min distance to serve first i houses with m mailboxes
        dp = [[float('inf')]*(n+1) for _ in range(k+1)]
        dp[0][0] = 0 # 0 post offices for 0 house

        for m in range(1, k+1): # 1, k+1 to support dp
            for i in range(1, n+1):
                for j in range(i):
                    dp[m][i] = min(dp[m][i], dp[m-1][j] + cost[j][i-1])
        
        return dp[k][n]


In [ ]:
# n - Top down DP - converting from bottom up
# TC - O(nlogn + n^3 + k*n^2) ~ O(k*n^2) as dp is dominant
# SC - O(k*n) # k postoffice into n houses

from functools import lru_cache

class Solution:
    def minDistance(self, houses, k: int) -> int:

        n = len(houses)
        houses.sort() # nlogn 
        # cost[i][j] = min distance to serve houses[i..j] with ONE mailbox
        cost = [[0]*n for _ in range(n)]
        # Precompute costs using median property
        for i in range(n):
            for j in range(i, n):
                m = (i+j)//2
                cost[i][j] = sum(abs(houses[x]-houses[m]) for x in range(i, j+1))

        # Step 2 -
        @lru_cache(None)
        def dfs(i, m):
            if i == n and m == 0:
                return 0
            # either reached the end of houses and postoffices are left
            # or post offices are over but all houses are not covered
            if i == n or n == 0:
                return float('inf')
            
            res = float('inf')
            for j in range(i, n):
                res = min(res, dfs(i+1, m-1) + cost[i][j])
            
            return res

        return dfs(0, k)


In [ ]:
# TC - 0(k * n^2)
# SC - O(n^2 + k*n)
# Decent Explanation - https://www.youtube.com/watch?v=ECcJUqdumIo
# GPT Explanation - https://chatgpt.com/c/694f8d86-09cc-8320-8eb3-fa941350fc69

from typing import List
from functools import lru_cache

class Solution:
    def minDistance(self, houses: List[int], k: int) -> int:
        # Sort houses so distance calculations make sense
        houses.sort()
        n = len(houses)

        # --------------------------------------------------
        # Step 1: Precompute cost[i][j]
        # cost[i][j] = minimum distance if ONE mailbox
        # serves houses from index i to j (inclusive)
        # --------------------------------------------------
        cost = [[0] * n for _ in range(n)]

        for i in range(n):
            for j in range(i, n):
                # Median minimizes sum of absolute distances
                mid = (i + j) // 2

                # Sum distance of each house in [i..j]
                # from the median house
                cost[i][j] = sum(
                    abs(houses[t] - houses[mid])
                    for t in range(i, j + 1)
                )

        # --------------------------------------------------
        # Step 2: Dynamic Programming with memoization
        #
        # dp(m, i) = minimum cost to cover houses
        # from index i to end using m mailboxes
        # --------------------------------------------------

        @lru_cache(None)
        def dp(m, i):
            # If all houses are covered
            if i == n:
                # Valid only if no mailboxes left
                return 0 if m == 0 else float('inf')

            # If houses remain but no mailboxes left
            if m == 0:
                return float('inf')

            res = float('inf')

            # Try placing one mailbox covering houses i..j
            # and solve remaining houses with m-1 mailboxes
            for j in range(i, n):
                res = min(
                    res,
                    cost[i][j] + dp(m - 1, j + 1)
                )

            return res

        # Start with k mailboxes covering from index 0
        return dp(k, 0)
